In [1]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

[W 2024-02-21 15:08:17.660 ServerApp] A `_jupyter_server_extension_points` function was not found in notebook_shim. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[I 2024-02-21 15:08:17.662 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2024-02-21 15:08:17.668 ServerApp] jupyter_server_terminals | extension was successfully linked.
[W 2024-02-21 15:08:17.671 LabApp] 'iopub_data_rate_limit' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2024-02-21 15:08:17.675 ServerApp] ServerApp.iopub_data_rate_limit config is deprecated in 2.0. Use ZMQChannelsWebsocketConnection.iopub_data_rate_limit.
[I 2024-02-21 15:08:17.675 ServerApp] jupyterlab | extension was successfully linked.
[I 2024-02-21 15:08:17.682 ServerApp] notebook | extension was successfully linked.
[I 2024-

# Machine Learning Project - PART A

## Read Data

In [57]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler


df = pd.read_pickle(r'/Users/maya/Documents/Information Systems/שנה ג׳/למידת מכונה/פרוייקט קורס/XY_train.pkl')
df.head()

,textID,text,sentiment,message_date,account_creation_date,previous_messages_dates,date_of_new_follower,date_of_new_follow,email,gender,email_verified,blue_tick,embedded_content,platform
0,b8f4e560fa,but yeah i like purple maybe thats why!! ;) ...,positive,2023-10-2 7:41:52,2015-4-13 0:1:16,"[2019-2-15 16:12:42, 2020-6-2 16:49:53, 2021-4...","[2016-1-13 7:12:26, 2016-5-17 13:40:38, 2016-1...","[2016-7-15 0:3:50, 2016-4-10 22:31:8, 2016-10-...",ClareRosindill3265@messenger.gov,F,NaN,False,mp4,facebook
1,f81a1511b2,_Henrie haha i WISH i coudl meet you.. you sho...,positive,2022-11-17 2:1:57,2013-7-1 18:58:16,"[2021-7-5 21:53:48, 2020-12-12 9:7:50, 2020-9-...","[2016-10-4 7:49:46, 2016-6-10 7:47:39, 2016-4-...","[2016-7-5 20:23:57, 2016-5-3 15:9:51, 2018-11-...",ErenaAntonchik231@messenger.gov,M,True,True,False,tiktok
2,3e9e3f0d69,"nah, you`re just altered forever Enjoy.",positive,2022-6-1 6:53:18,2013-8-8 2:21:12,"[2021-7-11 9:59:36, 2020-2-18 6:38:45, 2021-5-...","[2018-7-13 17:0:3, 2016-2-13 17:8:12, 2016-2-1...","[2016-7-9 22:15:10, 2018-4-13 20:37:55, 2016-1...",GodardCowlas6687@api.gov,F,True,True,False,tiktok
3,a068b95bd9,Can you ask Ryan why he stopped following me ...,negative,2022-1-4 11:38:57,2015-10-16 14:51:19,"[2021-8-16 4:55:32, 2021-6-1 10:13:37, 2021-12...",[2018-7-1 5:9:39],"[2018-10-18 7:23:20, 2018-4-10 12:17:6, 2016-6...",RalfStolberger6467@python.gov,M,False,False,mp4,tiktok
4,e6da2d1835,New issue of in the office...desperately want...,negative,2022-5-11 21:56:27,2013-12-15 19:8:40,"[2021-9-11 10:55:6, 2021-5-6 12:1:54, 2021-1-1...","[2018-11-2 11:16:12, 2018-10-18 16:8:34]","[2018-4-18 20:56:34, 2018-7-1 14:56:33, 2016-4...",MadelLeaton248@bbc.gov.il,M,False,False,None,None


## EDA Code

## Dataset Creation 

### 1. Pre-Processing

In [58]:
#------------ HANDLE MISSING VALUES ---------------#

# Calculate missing values before deletion
missing_values_before = df.isnull().sum()
df = df.dropna(thresh=df.shape[1]-2)
missing_values_after = df.isnull().sum()
missing_values_table = pd.DataFrame({
    'Before': missing_values_before,
    'After': missing_values_after,
    'Difference': missing_values_before - missing_values_after
})
print("Missing Values Before and After Deletion:")
print(missing_values_table)

# fill email with unknown
df['email'] = df['email'].fillna('unknown')
missing_values_count = df['email'].isnull().sum()

# fill embedded_content and platform by to their probability
embedded_content_prob = df['embedded_content'].value_counts(normalize=True)
platform_prob = df['platform'].value_counts(normalize=True)

def impute_missing_values(row, prob_dist):
    if pd.isnull(row):
        return np.random.choice(prob_dist.index, p=prob_dist.values)
    else:
        return row
df['embedded_content'] = df['embedded_content'].apply(lambda x: impute_missing_values(x, embedded_content_prob))
df['platform'] = df['platform'].apply(lambda x: impute_missing_values(x, platform_prob))

# fill missing values for email_verified and blue_tick
df['email_verified'].fillna(df['blue_tick'], inplace=True)
df['blue_tick'].fillna(df['email_verified'], inplace=True)

# fill gender randomly
df['gender'].replace('None', np.nan, inplace=True)
gender_counts = df['gender'].value_counts()
df['gender'].fillna(pd.Series(np.random.choice(gender_counts.index,
                                               size=len(df.index),
                                               p=(gender_counts / gender_counts.sum()))),
                    inplace=True)

# delete the remaining rows with missing values
df = df.dropna()

#------------ HANDLE DATA CONVERSIONS ---------------#

# convert message_date to categorical
df['message_date'] = pd.to_datetime(df['message_date'])
df['hour'] = df['message_date'].dt.hour
morning_interval = range(6, 12)  # 6:00 AM to 11:59 AM
noon_interval = range(12, 18)     # 12:00 PM to 5:59 PM
evening_interval = range(18, 24)  # 6:00 PM to 11:59 PM
def categorize_hour(hour):
    if hour in morning_interval:
        return 'Morning'
    elif hour in noon_interval:
        return 'Noon'
    elif hour in evening_interval:
        return 'Evening'
    else:
        return 'Night'
df['message_time_category'] = df['hour'].apply(categorize_hour)
df.drop(columns=['hour'], inplace=True)

#------------ HANDLE CHANGES FOR N-GRAM  ---------------#

# lower case
lower = df['text'].apply(str.lower)


# stemming
from nltk.stem import SnowballStemmer
stem = SnowballStemmer('english')
stemmed = lower.apply(lambda x: ' '.join(stem.stem(word) for word in str(x).split()))
    

# remove punctuation
import re
rem_punc = stemmed.apply(lambda x: re.sub(r'[^\w\s]',' ',x))


# remove numbers
rem_num = rem_punc.apply(lambda x: " ".join(x for x in x.split() if not x.isdigit()))

# remove words with 1 letter
rem_length1 = rem_num.apply(lambda x: re.sub(r'\b\w{1}\b',' ',x))

# remove top 0.05% of most common or not common words
h_pct = 0.05
l_pct = 0.05

# remove the top $h_pct of the most frequent words 
high_freq = pd.Series(''.join(rem_length1).split()).value_counts()[:int(pd.Series(' '.join(rem_length1).split()).count()*h_pct/100)]

rem_high = rem_length1.apply(lambda x: " ".join(x for x in x.split() if x not in high_freq))
    
# remove the top $l_pct of the least frequent words
low_freq = pd.Series(''.join(rem_high).split()).value_counts()[:-int(pd.Series(' '.join(rem_high).split()).count()*l_pct/100):-1]

rem_low = rem_high.apply(lambda x: " ".join(x for x in x.split() if x not in low_freq))

# remove double spaces
rem_punc = rem_low.apply(lambda x: re.sub(r'[^\w\s]',' ',x))

df['clean_text'] = rem_punc


# Calculate missing values before deletion
missing_values_before = df.isnull().sum()
df = df.dropna(thresh=df.shape[1]-2)
missing_values_after = df.isnull().sum()
missing_values_table = pd.DataFrame({
    'Before': missing_values_before,
    'After': missing_values_after,
    'Difference': missing_values_before - missing_values_after
})
print("Missing Values Before and After Deletion:")
print(missing_values_table)



Missing Values Before and After Deletion:
                         Before  After  Difference
textID                        0      0           0
text                          0      0           0
sentiment                     0      0           0
message_date                  0      0           0
account_creation_date         0      0           0
previous_messages_dates       0      0           0
date_of_new_follower          0      0           0
date_of_new_follow            0      0           0
email                       891    825          66
gender                     1619   1511         108
email_verified             1023    941          82
blue_tick                  1439   1354          85
embedded_content            906    854          52
platform                    750    697          53


/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_24852/1259465266.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['email_verified'].fillna(df['blue_tick'], inplace=True)
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_24852/1259465266.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

Missing Values Before and After Deletion:
                         Before  After  Difference
textID                        0      0           0
text                          0      0           0
sentiment                     0      0           0
message_date                  0      0           0
account_creation_date         0      0           0
previous_messages_dates       0      0           0
date_of_new_follower          0      0           0
date_of_new_follow            0      0           0
email                         0      0           0
gender                        0      0           0
email_verified                0      0           0
blue_tick                     0      0           0
embedded_content              0      0           0
platform                      0      0           0
message_time_category         0      0           0
clean_text                    0      0           0


### 3. Feature Extraction

In [59]:
#1. create a new column based on the length of messages
df['message_length'] = df['text'].apply(lambda x: len(x))

#2. create the number of messages sent by the user
df['num_messages_sent'] = df['previous_messages_dates'].apply(len)

#3. create num of followers and following
df['follower_count'] = df['date_of_new_follower'].apply(lambda x: len(x))
df['following_count'] = df['date_of_new_follow'].apply(lambda x: len(x))

#4. append new columns created to the df 
new_columns_df = df[['follower_count', 'following_count']]

#5. N-GRAM

X = df['clean_text']
Y = df['sentiment']

ngram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features = 100)
X_ngrams = ngram_vectorizer.fit_transform(X).toarray()

# Check words frequency
# sum_of_words = X_ngrams.sum(axis = 0)
# words_freq = [(word, sum_of_words[i]) for word, i in ngram_vectorizer.vocabulary_.items()]
# words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
# 
# print(words_freq)

df_output = pd.DataFrame(data = X_ngrams, columns = ngram_vectorizer.get_feature_names_out())


#6. extract email domain endings
def extract_email_domain_ending(email):
    if pd.isnull(email):
        return 'Missing'
    match = re.search(r'\.(\w+)$', email)
    if match:
        return match.group(1)
    else:
        return 'Unknown'
df['email_domain_ending'] = df['email'].apply(extract_email_domain_ending)
email_domain_ending_counts = df['email_domain_ending'].value_counts()

#7. create seniority in years
df['account_creation_date'] = pd.to_datetime(df['account_creation_date'])
current_date = datetime.now()
df['seniority'] = (current_date - df['account_creation_date']).dt.days / 365.25

#8. create the average time difference between messages
def calculate_average_time_difference(message_dates_array):
    message_dates = [datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S') for date_str in message_dates_array]
    time_diffs = []
    for i in range(1, len(message_dates)):
        time_diff = (message_dates[i] - message_dates[i - 1]).total_seconds()
        if time_diff < 0:
            time_diff = (message_dates[i - 1] - message_dates[i]).total_seconds()
        time_diffs.append(time_diff)
    if len(time_diffs) > 0:
        average_time_difference = sum(time_diffs) / len(time_diffs)
        average_time_difference = int(average_time_difference)
        return average_time_difference
    else:
        return None
df['average_time_difference'] = df['previous_messages_dates'].apply(calculate_average_time_difference)

### 4. Feature Representation

In [60]:
#1. normalize the values by dividing each column by its maximum value
columns_to_normalize = ['message_length', 'num_messages_sent', 'follower_count', 'following_count', 'seniority']
for column in columns_to_normalize:
    max_value = df[column].max()
    df[f'normalized_{column}'] = df[column] / max_value
for column in columns_to_normalize:
    normalized_column_name = f'normalized_{column}'
    # print(f"Normalized {column}:")
    # print(df[normalized_column_name])

#2. one-hot coding
email_domain_ending_onehot = pd.get_dummies(df['email_domain_ending'], prefix='email_ending')
embedded_content_onehot = pd.get_dummies(df['embedded_content'], prefix='embedded_content')
platform_onehot = pd.get_dummies(df['platform'], prefix='platform')
message_time_category_onehot = pd.get_dummies(df['message_time_category'], prefix='message_time')
df = pd.concat([df, email_domain_ending_onehot, embedded_content_onehot, platform_onehot, message_time_category_onehot], axis=1)
df.drop(['email_domain_ending', 'embedded_content', 'platform', 'message_time_category'], axis=1, inplace=True)

#3. convert one hot coding to binary values
bool_to_binary = {True: 1, False: 0}
df['email_verified'] = df['email_verified'].map(bool_to_binary)
df['blue_tick'] = df['blue_tick'].map(bool_to_binary)

for column in df.columns:
    if df[column].dtype == bool:
        df[column] = df[column].astype(int) 

gender_to_binary = {'F': 1, 'M': 0}
df['gender'] = df['gender'].map(gender_to_binary)

#4. normalization for the n-gram features
scaler = MinMaxScaler()
X_ngrams_normalized = scaler.fit_transform(X_ngrams)
df_normalized = pd.DataFrame(X_ngrams_normalized, columns=ngram_vectorizer.get_feature_names_out())
df = pd.concat([df.reset_index(drop=True), df_normalized], axis=1)


#6. arrange data again 
sentiment_mapping = {'positive': 1, 'negative': -1}
df['sentiment'] = df['sentiment'].map(sentiment_mapping)
df = df.drop(columns=['text', 'previous_messages_dates', 'message_date',
                      'email', 'date_of_new_follower', 'date_of_new_follow',
                      'account_creation_date', 'message_length', 'num_messages_sent',
                      'follower_count', 'following_count', 
                      'seniority', 'clean_text', 'average_time_difference'])

df.head()


,textID,sentiment,gender,email_verified,blue_tick,normalized_message_length,normalized_num_messages_sent,normalized_follower_count,normalized_following_count,normalized_seniority,email_ending_Unknown,email_ending_com,email_ending_de,email_ending_edu,email_ending_gov,email_ending_il,email_ending_jp,email_ending_ke,email_ending_org,email_ending_ru,embedded_content_False,embedded_content_jpeg,embedded_content_link,embedded_content_mp4,platform_facebook,platform_instagram,platform_telegram,platform_tiktok,platform_whatsapp,platform_x,message_time_Evening,message_time_Morning,message_time_Night,message_time_Noon,after,again,an,as,awesom,bad,bed,been,best,better,bit,by,cant,come,cool,could,did,didn,dont,down,enjoy,even,ever,final,follow,friend,fun,girl,gonna,guy,haha,has,hate,he,her,here,him,home,how,hurt,if,last,ll,lol,long,man,mom,more,morn,need,new,next,nice,off,oh,onli,or,over,peopl,right,sad,say,school,she,should,show,sick,sleep,sorri,start,still,sure,take,than,thank,them,then,there,thing,though,tomorrow,tonight,tri,tweet,twitpic,twitter,veri,wait,watch,way,week,weekend,well,were,when,whi,who,wish,would,yeah
0,b8f4e560fa,1,1,0,0,0.361702,0.244898,0.897959,0.285714,0.795477,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5
1,f81a1511b2,1,0,1,1,0.843972,0.979592,0.367347,0.857143,0.955261,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
2,3e9e3f0d69,1,1,1,1,0.297872,0.612245,0.306122,0.489796,0.946165,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,a068b95bd9,-1,0,0,0,0.397163,0.102041,0.020408,0.081633,0.749754,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0
4,e6da2d1835,-1,0,0,0,0.751773,0.163265,0.040816,0.122449,0.914208,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [61]:
# nan_rows = df[df['normalized_average_time_difference'].isna()]
# nan_rows


In [62]:
# Separate the dataset into positive and negative sentiment groups
positive_sentiment = df[df['sentiment'] == 1]
negative_sentiment = df[df['sentiment'] == -1]

# Remove 'sentiment', 'textID' columns as we are not considering it as a feature
features = df.drop(columns=['sentiment', 'textID']).columns

# print(features)

fisher_scores = {}

# Calculate Fisher score for each feature
for feature in features:
    mean_positive = positive_sentiment[feature].mean()
    mean_negative = negative_sentiment[feature].mean()
    var_positive = positive_sentiment[feature].var()
    var_negative = negative_sentiment[feature].var()

    # print(var_negative)
    # print(var_positive)

    fisher_score = ((mean_positive - mean_negative)**2) / (var_positive + var_negative)
    # print(feature)
    fisher_scores[feature] = round(fisher_score, 3)  # Round to 3 decimal places

# print(fisher_scores)

fisher_df = pd.DataFrame.from_dict(fisher_scores, orient='index', columns=['Fisher Score'])

# sort the features by highest to lowest
fisher_df = fisher_df.sort_values(by='Fisher Score', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

fisher_df

,Fisher Score
normalized_num_messages_sent,1.868
normalized_follower_count,1.862
normalized_following_count,1.753
email_verified,0.203
blue_tick,0.168
gender,0.156
platform_instagram,0.133
normalized_seniority,0.101
thank,0.074
embedded_content_False,0.062


In [64]:
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np

# Assuming 'X_cat' is your feature matrix and 'Y' is your target variable

# Extract the target variable 'sentiment'
Y = df['sentiment']

# Drop the 'textID' column from the feature DataFrame
X_cat = df.drop(columns=['textID', 'sentiment'])

# Initialize SelectKBest with chi-squared as the scoring function and k=15
chi2_features = SelectKBest(chi2, k=15)

# Fit SelectKBest to the data and transform it
X_cat_kbest = chi2_features.fit_transform(X_cat, Y)

# Get the p-values of the features and round to 4 digits
p_values = chi2_features.pvalues_
p_values_rounded = np.round(p_values, 6)

# Get the selected features
selected_features = X_cat.columns[chi2_features.get_support()]

# Convert the array of rounded p-values into a DataFrame
p_values_df = pd.DataFrame({'Feature': X_cat.columns, 'P-value': p_values_rounded})

# Filter the selected features based on the support
selected_features_df = pd.DataFrame({'Feature': selected_features})

# Print the DataFrames
print("P-values DataFrame:")
print(p_values_df)

print("\nSelected Features DataFrame:")
selected_features_df




P-values DataFrame:
                          Feature   P-value
0                          gender  0.000000
1                  email_verified  0.000000
2                       blue_tick  0.000000
3       normalized_message_length  0.784816
4    normalized_num_messages_sent  0.000000
5       normalized_follower_count  0.000000
6      normalized_following_count  0.000000
7            normalized_seniority  0.039884
8            email_ending_Unknown  0.429138
9                email_ending_com  0.171366
10                email_ending_de  0.000000
11               email_ending_edu  0.000000
12               email_ending_gov  0.270472
13                email_ending_il  0.000000
14                email_ending_jp  0.073445
15                email_ending_ke  0.000000
16               email_ending_org  0.102219
17                email_ending_ru  0.943974
18         embedded_content_False  0.000000
19          embedded_content_jpeg  0.000000
20          embedded_content_link  0.000000
21          

,Feature
0,gender
1,email_verified
2,blue_tick
3,normalized_num_messages_sent
4,normalized_follower_count
5,normalized_following_count
6,email_ending_edu
7,embedded_content_False
8,embedded_content_link
9,embedded_content_mp4
